In [1]:
import numpy as np
import pandas as pd
from rake_nltk import Rake
import ast

from gensim.models.doc2vec import Doc2Vec
from sentence_transformers import SentenceTransformer

import gensim
from gensim.models import Word2Vec 
from gensim.models import KeyedVectors
from nltk.tokenize import RegexpTokenizer
from nltk import sent_tokenize
import re
from tqdm import tqdm
import json
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

from sklearn.metrics import pairwise_distances
from collections import defaultdict, Counter

stop_words = set(stopwords.words('english'))

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

ModuleNotFoundError: No module named 'rake_nltk'

In [ ]:
def extract_keywords(text, num_keywords):
    r = Rake()
    r.extract_keywords_from_text(text)
    return r.get_ranked_phrases()[:num_keywords]

def get_sentence_embeddings(sentence, new_model, base_model):
    word_embeddings = []
    for word in set(sentence):
        if word in new_model:
            word_emb = list(new_model[word])
            word_embeddings.append(word_emb)
        elif word in base_model:
            word_emb = list(base_model[word]) 
            word_embeddings.append(word_emb)
    return word_embeddings

def tokenize(text, stopwords, max_len = 20):
    return [token for token in gensim.utils.simple_preprocess(text, max_len=max_len) if token not in stopwords]

def tokenize_sentence(sentence):
    tokenizer = RegexpTokenizer(r'\w+')
    
    sentence = tokenizer.tokenize(sentence)
    sentence = [re.sub(r'\[.*?\]', '', re.sub(r'\d+', '', w)).lower() for w in sentence]
    
    return [w for w in sentence if not w in stop_words]

def sort_dict(sent_dict, topn):
    final_sentences = []
    dct = defaultdict(list)

    for k, v in sent_dict.items():
        dct[v].append(k)

    for k, v in sorted(dct.items(), reverse=True)[:topn]:
        final_sentences.append(v)
    return final_sentences


def get_sentence_embedding(sentence, new_model, base_model):
    word_embeddings = []
    sentence =  tokenize_sentence(sentence)
    
    for word in set(sentence):
        if word in new_model:
            word_emb = list(new_model[word])
            word_embeddings.append(word_emb)
        elif word in base_model:
            word_emb = list(base_model[word]) 
            word_embeddings.append(word_emb)
            
    sentence_emb_matrix = np.array(word_embeddings)
    sentence_embedding = []
    
    if sentence_emb_matrix.shape[0] == 0:
        sentence_embedding = list(np.random.uniform(-1, 1, 300))
    else:
        sentence_embedding = list(np.mean(sentence_emb_matrix, axis = 0))
        
    return sentence_embedding


def summarize_section(paper, section_name, section_sentences, embeddings, embedding_type, models):
    if section_name == 'abstract':
        keywords = [paper['title']]
    else:
        keywords = [section_name]     
    keywords += paper['keywords']
    keywords = list(set(keywords))
    
    custom_keywords = list(set(paper['additional_keywords']))
        
    sentences = section_sentences
    summary_len = np.ceil(len(sentences) / 3)
    
    section_embeddings = [embeddings[i] for i in sentences]
    
    if len(sentences) <= 1:
        return None, None, 1   
    if embedding_type == 'word2vec_emb':
        keywords_embeddings = get_word2vec_embs(keywords, models)
        cust_keywords_embeddings = get_word2vec_embs(custom_keywords, models)
              
    elif embedding_type == 'sent_transf_emb':
        keywords_embeddings = get_sent_transf_embs(keywords, models[0])
        cust_keywords_embeddings = get_sent_transf_embs(custom_keywords, models[0])
        
    else:
        print('Invalid Embedding type!')
        return

    keywords_section_dist = pairwise_distances(section_embeddings, keywords_embeddings)
    cust_keywords_section_dist = pairwise_distances(section_embeddings, cust_keywords_embeddings)
    
    dist_df = pd.DataFrame(keywords_section_dist, columns = keywords)
    dist_df['sentence'] = sentences
    
    cust_dist_df = pd.DataFrame(cust_keywords_section_dist, columns = custom_keywords)
    cust_dist_df['sentence'] = sentences 
    return dist_df, cust_dist_df, summary_len
    


def build_summary(dist_df, cust_dist_df, summary_len, method = 'closeness'):
    sentences = []
    values = []

    for column in dist_df.iloc[:, :-1]:
        filt_df = dist_df.sort_values(column).iloc[:3]

        for _, row in filt_df.iterrows():
            sentences.append(row['sentence'])
            values.append(row[column])

    sent_dist_df = pd.DataFrame({'sentence': sentences, 'distance': values})

    sentences = []
    values = []

    for column in cust_dist_df.iloc[:, :-1]:
        filt_df = cust_dist_df.sort_values(column).iloc[:1]

        for _, row in filt_df.iterrows():
            sentences.append(row['sentence'])
            values.append(row[column])

    sent_dist_df = pd.concat([sent_dist_df, pd.DataFrame({'sentence': sentences, 'distance': values})], axis = 0)
    
    if method == 'closeness':
        summary = sent_dist_df.drop_duplicates('sentence').sort_values('distance').iloc[:summary_len]['sentence'].values
        
    elif method == 'frequency':
        summary = [i[0] for i in sort_dict(dict(Counter(sent_dist_df.sentence.values)), summary_len)]
    
    else:
        print('Invalid method. Exiting...')
        return
    
    sorted_summary = dist_df[dist_df.sentence.isin(summary)].sort_index()['sentence'].values
    
    return sorted_summary

def summarize_paper(paper, sent_df, embedding_col, extraction_method, models):
    paper_sent_df = sent_df[sent_df.paper_id == paper.paper_id]
    summary_sections = {}
    sentences = paper_sent_df.sentence.values
    embeddings = paper_sent_df[embedding_col].values
    
    sent_emb_dict = dict(zip(sentences, embeddings))
    
    all_sections = ['abstract'] + list(paper.sections.keys())
    
    for section_name in all_sections:
        if section_name != 'abstract' and paper.sections[section_name] == '':
            summary = ''
            summary_sections[section_name] = summary
        else:
            section_sentences = paper_sent_df[paper_sent_df.section == section_name].sentence.values
            dist_df, cust_dist_df, summary_len = summarize_section(paper, section_name, section_sentences, sent_emb_dict, 
                                                                   embedding_col, models)

            if dist_df is None:
                summary = [paper.sections[section_name]]
            else:
                summary = build_summary(dist_df, cust_dist_df, int(summary_len), method = extraction_method)

            summary_sections[section_name] = ' '.join(summary)
                    
    return summary_sections


def get_doc2vec_embs(sentences, model):
    doc2vec_embeddings = []
    for sentence in sentences:
        test_sent = tokenize(sentence.lower(), [])
        test_sent_vector = model.infer_vector(test_sent)
        doc2vec_embeddings.append(test_sent_vector)
        
    return doc2vec_embeddings


def get_sent_transf_embs(sentences, model):
    return model.encode(sentences)

def get_word2vec_embs(sentences, models):
    return [get_sentence_embedding(i, models[0], models[1]) for i in sentences]

In [ ]:
base_model = KeyedVectors.load_word2vec_format('../Fine-Tuning-Word2Vec-Embeddings/GoogleNews-vectors-negative300.bin', 
                                               binary=True)
new_model = KeyedVectors.load_word2vec_format("../Fine-Tuning-Word2Vec-Embeddings/new_word2vec.model")

INFO - 01:42:36: loading projection weights from ../Fine-Tuning-Word2Vec-Embeddings/GoogleNews-vectors-negative300.bin
INFO - 01:44:41: KeyedVectors lifecycle event {'msg': 'loaded (3000000, 300) matrix of type float32 from ../Fine-Tuning-Word2Vec-Embeddings/GoogleNews-vectors-negative300.bin', 'binary': True, 'encoding': 'utf8', 'datetime': '2021-05-24T01:44:27.058507', 'gensim': '4.0.1', 'python': '3.7.1 (default, Dec 10 2018, 22:54:23) [MSC v.1915 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'load_word2vec_format'}
INFO - 01:45:23: loading projection weights from ../Fine-Tuning-Word2Vec-Embeddings/new_word2vec.model
INFO - 01:46:01: KeyedVectors lifecycle event {'msg': 'loaded (29222, 300) matrix of type float32 from ../Fine-Tuning-Word2Vec-Embeddings/new_word2vec.model', 'binary': False, 'encoding': 'utf8', 'datetime': '2021-05-24T01:46:01.430812', 'gensim': '4.0.1', 'python': '3.7.1 (default, Dec 10 2018, 22:54:23) [MSC v.1915 64 bit (AMD64)]', 'platform': '

In [ ]:
test_df = pd.read_csv('../data/test_papers_word2vec.csv')
test_df.sections = test_df.sections.apply(ast.literal_eval)
test_df.sentences = test_df.sentences.apply(ast.literal_eval)
test_df.sentence_embeddings = test_df.sentence_embeddings.apply(ast.literal_eval)
test_df.keywords = test_df.keywords.apply(ast.literal_eval)

paper_ids = test_df.paper_id.values

sent_df = pd.read_csv('../data/test_sentence_embeddings.csv')

sent_df['doc2vec_emb'] = sent_df['doc2vec_emb'].apply(ast.literal_eval)
sent_df['sent_transf_emb'] = sent_df['sent_transf_emb'].apply(ast.literal_eval)
sent_df['word2vec_emb'] = sent_df['word2vec_emb'].apply(ast.literal_eval)

additional_keywords = []

for _, row in test_df.iterrows():
    new_keywords_dict = {}
    num_abstract_sent = sent_df[(sent_df.paper_id == row['paper_id']) & (sent_df.section == 'abstract')].shape[0]
    new_keywords_dict['abstract'] = extract_keywords(row['abstract'], num_abstract_sent // 3)
    
    for key, value in row['sections'].items():
        new_keywords_dict[key] = [key]
        num_section_sent = sent_df[(sent_df.paper_id == row['paper_id']) & (sent_df.section == key)].shape[0]
        new_keywords_dict[key] += extract_keywords(value, num_section_sent // 3)
    
    additional_keywords.append(new_keywords_dict)
        
test_df['additional_keywords'] = additional_keywords

In [ ]:
test_df.head()

,paper_id,category,name,num_pages,num_formulas,num_figures,title,keywords,abstract,sections,num_sentences,sent_by_page,partition,text,sentences,sentences_tokenized,sentence_embeddings,additional_keywords
0,paper_1,Artificial Intelligence,A Model for Clustering Social Media Data for E...,4,0,4,A Model for Clustering Social Media Data for E...,"[Social Media, Twitter Application Programming...","Through Social media, people are able to write...",{'Introduction': 'Clustering is a descriptive ...,76,25.333333,test,"Through Social media, people are able to write...","[Through Social media, people are able to writ...","[['social', 'media', 'people', 'able', 'write'...","[[0.25343436, 0.012410111, -0.34438372, 0.3019...",{'abstract': ['sentiments using various social...
1,paper_2,Artificial Intelligence,An Intelligent System for Traffic Control in S...,8,5,9,An Intelligent System for Traffic Control in S...,"[Smart Cities, Traffic Congestion, Intelligent...",Current traffic light systems use a fixed time...,"{'Introduction': '', 'Background': 'Traffic co...",99,14.142857,test,Current traffic light systems use a fixed time...,[Current traffic light systems use a fixed tim...,"[['current', 'traffic', 'light', 'systems', 'u...","[[0.08013701, -0.043167926, 0.019797344, 0.273...",{'abstract': ['density based traffic control a...
2,paper_3,Artificial Intelligence,Architecture Trends of Adaptive Educational Hy...,14,0,12,Architecture Trends of Adaptive Educational Hy...,"[Adaptive Educational Hypermedia Systems, Arch...",The aim of this article is to present the gene...,{'Introduction': 'Adaptive Hypermedia Educatio...,203,15.615385,test,The aim of this article is to present the gene...,[The aim of this article is to present the gen...,"[['aim', 'article', 'present', 'general', 'arc...","[[0.13080898, -0.11806064, -0.047516536, 0.097...",{'abstract': ['based adaptive educational hype...
3,paper_21,Computer Science and Technology,A New Powerful Scheme Based on Self Invertible...,5,1,2,A New Powerful Scheme Based on Self Invertible...,"[Minimum Distance, Minimum Weight, BCH Codes, ...","In this paper, we present the powerful scheme ...",{'Introduction': 'In telecommunication and sto...,60,15.000000,test,"In this paper, we present the powerful scheme ...","[In this paper, we present the powerful scheme...","[['paper', 'present', 'powerful', 'scheme', 'z...","[[0.22547568, 0.05606357, -0.1207298, 0.160863...",{'abstract': ['zimmermann self invertible stab...
4,paper_31,Computer Science and Technology,Design and Implementation of Intelligent Medic...,6,2,10,Design and Implementation of Intelligent Medic...,"[Smart Medical Care, ZigBee, Semantic Matching]",With the continuous improvement of human livin...,"{'Introduction': 'From 1990 to 2017, the morbi...",124,24.800000,test,With the continuous improvement of human livin...,[With the continuous improvement of human livi...,"[['continuous', 'improvement', 'human', 'livin...","[[-0.21172313, 0.35368863, -0.16834831, 0.2698...",{'abstract': ['innovatively proposes semantic ...


In [ ]:
paper = 'paper_1'

summarize_paper(paper_id, sent_df, embedding_col, 'closeness')

{'abstract': 'This research is aimed at tackling the afore-mentioned problems by adopting a density based traffic control approach using Jakpa Junction, one of the busiest junctions in Delta State, Nigeria as a case study. The developed system uses a microcontroller of PIC89C51 microcontroller duly interfaced with sensors. The developed system has proven to be smart and intelligent and capable of curbing incidences of traffic malpractices and inefficiencies that have been the bane of current traffic congestion control systems in emerging cities of the third world.',
 'Introduction': '',
 'Background': 'These include loss of man-hours, accident, missed opportunities, noise pollution, air-pollution, increased fuel consumption, increased tendency to violate traffic rules, and in some cases extortion by corrupt traffic control officials. Conventional traffic light system is based on fixed time concept allotted to each side of the junction which cannot be varied as per varying traffic densi

In [ ]:
paper = 'paper_2'

summarize_paper(paper_id, sent_df, embedding_col, 'frequency')

{'abstract': 'This research is aimed at tackling the afore-mentioned problems by adopting a density based traffic control approach using Jakpa Junction, one of the busiest junctions in Delta State, Nigeria as a case study. The sensors used in this project were infra-red (IR) sensors and photodiodes which were placed in a Line of Sight configuration across the loads to detect the density of the traffic signal. The developed system has proven to be smart and intelligent and capable of curbing incidences of traffic malpractices and inefficiencies that have been the bane of current traffic congestion control systems in emerging cities of the third world.',
 'Introduction': '',
 'Background': 'These include loss of man-hours, accident, missed opportunities, noise pollution, air-pollution, increased fuel consumption, increased tendency to violate traffic rules, and in some cases extortion by corrupt traffic control officials. Conventional traffic light system is based on fixed time concept a

In [ ]:
doc2vec_model = Doc2Vec.load('../doc2vec/doc2vec.model')
sent_transf_model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

INFO - 04:30:54: loading Doc2Vec object from ../doc2vec/doc2vec.model
INFO - 04:30:54: loading dv recursively from ../doc2vec/doc2vec.model.dv.* with mmap=None
INFO - 04:30:54: loading wv recursively from ../doc2vec/doc2vec.model.wv.* with mmap=None
INFO - 04:30:54: setting ignored attribute cum_table to None
INFO - 04:30:55: Doc2Vec lifecycle event {'fname': '../doc2vec/doc2vec.model', 'datetime': '2021-05-23T04:30:55.359147', 'gensim': '4.0.1', 'python': '3.7.1 (default, Dec 10 2018, 22:54:23) [MSC v.1915 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'loaded'}
INFO - 04:30:55: Load pretrained SentenceTransformer: distilbert-base-nli-stsb-mean-tokens
INFO - 04:30:55: Did not find folder distilbert-base-nli-stsb-mean-tokens
INFO - 04:30:55: Search model on server: http://sbert.net/models/distilbert-base-nli-stsb-mean-tokens.zip
INFO - 04:30:55: Load SentenceTransformer from folder: C:\Users\User/.cache\torch\sentence_transformers\sbert.net_models_distilbert-base-n

In [ ]:
summarize_paper(paper, sent_df, embedding_col, extraction_method, models)

## Doc2vec

In [ ]:
doc2vec_model = Doc2Vec.load('../doc2vec/doc2vec.model')


INFO - 02:21:03: loading Doc2Vec object from ../doc2vec/doc2vec.model
INFO - 02:21:03: loading dv recursively from ../doc2vec/doc2vec.model.dv.* with mmap=None
INFO - 02:21:03: loading wv recursively from ../doc2vec/doc2vec.model.wv.* with mmap=None
INFO - 02:21:03: setting ignored attribute cum_table to None
INFO - 02:21:04: Doc2Vec lifecycle event {'fname': '../doc2vec/doc2vec.model', 'datetime': '2021-05-24T02:21:04.251556', 'gensim': '4.0.1', 'python': '3.7.1 (default, Dec 10 2018, 22:54:23) [MSC v.1915 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'loaded'}


In [ ]:
tqdm.pandas()
test_df['summary_by_closeness'] = test_df.progress_apply(lambda x: 
                                                    summarize_paper(x, sent_df, 'doc2vec_emb', 'closeness', [doc2vec_model]),
                                                    axis = 1)

test_df['summary_by_frequency'] = test_df.progress_apply(lambda x: 
                                                    summarize_paper(x, sent_df, 'doc2vec_emb', 'frequency', [doc2vec_model]),
                                                    axis = 1)

100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [00:25<00:00,  1.40it/s]


In [ ]:
test_df[['paper_id', 'title', 'abstract', 'sections', 'summary_by_closeness', 
         'summary_by_frequency']].to_csv('../results/query_based_summ_doc2vec.csv', index = False)

## Sentence Transformer

In [ ]:
sent_transf_model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

INFO - 03:03:43: Load pretrained SentenceTransformer: distilbert-base-nli-stsb-mean-tokens
INFO - 03:03:43: Did not find folder distilbert-base-nli-stsb-mean-tokens
INFO - 03:03:43: Search model on server: http://sbert.net/models/distilbert-base-nli-stsb-mean-tokens.zip
INFO - 03:03:43: Load SentenceTransformer from folder: C:\Users\User/.cache\torch\sentence_transformers\sbert.net_models_distilbert-base-nli-stsb-mean-tokens
INFO - 03:03:46: Use pytorch device: cpu


In [ ]:
tqdm.pandas()
test_df['summary_by_closeness'] = test_df.progress_apply(lambda x: 
                                            summarize_paper(x, sent_df, 'sent_transf_emb', 'closeness', [sent_transf_model]),
                                            axis = 1)
test_df['summary_by_frequency'] = test_df.progress_apply(lambda x: 
                                            summarize_paper(x, sent_df, 'sent_transf_emb', 'frequency', [sent_transf_model]), 
                                            axis = 1)


  0%|                                                                                           | 0/36 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [02:39<00:00,  4.44s/it]


In [ ]:
test_df[['paper_id', 'title', 'abstract', 'sections', 'summary_by_closeness', 
         'summary_by_frequency']].to_csv('../results/query_based_summ_sent_transf.csv', index = False)

## Word2vec

In [ ]:
tqdm.pandas()
test_df['summary_by_closeness'] = test_df.progress_apply(lambda x: 
                                            summarize_paper(x, sent_df, 'word2vec_emb', 'closeness', [new_model, base_model]),
                                            axis = 1)
test_df['summary_by_frequency'] = test_df.progress_apply(lambda x: 
                                            summarize_paper(x, sent_df, 'word2vec_emb', 'frequency', [new_model, base_model]), 
                                            axis = 1)

100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [00:13<00:00,  2.59it/s]


In [ ]:
test_df[['paper_id', 'title', 'abstract', 'sections', 'summary_by_closeness', 
         'summary_by_frequency']].to_csv('../results/query_based_summ_word2vec.csv', index = False)